# Inicializacion
 Para empezar a preprocesar los datos, debemos cargar el dataset en cuestion para ello tenemos dos formas, la primera con el "archivo.csv" se carga manualmente habiendo descargado el dataset necesaria en local.

 La siguiente forma presentada a continuacion no es muy recomendable, ya que necesitas "autorizacion" de una API key de kaggle con las credenciales (archivo incluido junto con el notebook) pero para este caso haremos excepcion por motivos academicos.


## Instalacion de las dependencias

Procedemos a importar pandas para usar el dataframe, seguido de la instalacion de kaggle y por ultimo subir el archivo json adjuntado.

In [3]:
import pandas as pd

# Instalamos la libreria de kaggle
!pip install -q kaggle

# Subimos el archivo kaggle.json que contiene la API para descargar el dataset
from google.colab import files
files.upload()



Saving kaggle.json to kaggle.json


{'kaggle.json': b'{\r\n  "username": "juanlps",\r\n  "key": "KGAT_80c3ba9cd5c2b90ff6a819db05e94ed6"\r\n}\r\n\t\r\n'}

Accedemos al archivo json para que al momento de descargar el dataset las credenciales no den errores.

In [4]:
# Crear la carpeta donde la librería busca el token
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


Una vez teniendo las credenciales configuradas se descarga el dataset y se descomprime.

In [5]:
!kaggle datasets download -d alitaqi000/world-university-rankings-2023
!unzip -q world-university-rankings-2023.zip -d ./

Dataset URL: https://www.kaggle.com/datasets/alitaqi000/world-university-rankings-2023
License(s): ODbL-1.0
  0% 0.00/70.2k [00:00<?, ?B/s]
100% 70.2k/70.2k [00:00<00:00, 142MB/s]


# Verificacion del Dataset

Verificamos que el dataset se descargo, se descomprimio correctamente y se carga en la variable df usando pandas.

y usamos la funcion head() para que muestre las primeras 5 entradas.

In [6]:
df = pd.read_csv('World University Rankings 2023.csv')
df.head()

,University Rank,Name of University,Location,No of student,No of student per staff,International Student,Female:Male Ratio,OverAll Score,Teaching Score,Research Score,Citations Score,Industry Income Score,International Outlook Score
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,48 : 52,96.4,92.3,99.7,99.0,74.9,96.2
1,2,Harvard University,United States,"21,887",9.6,25%,50 : 50,95.2,94.8,99.0,99.3,49.5,80.5
2,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,47 : 53,94.8,90.9,99.5,97.0,54.2,95.8
3,3,Stanford University,United States,"16,164",7.1,24%,46 : 54,94.8,94.2,96.7,99.8,65.0,79.8
4,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,40 : 60,94.2,90.7,93.6,99.8,90.9,89.3


# Inicio del preprocesamiento

Verificamos cuantos valores nulos existen en los datos.

In [7]:
# Cuenta los valores nulos por columna
nulos_por_columna = df.isnull().sum()
print(nulos_por_columna)

University Rank                  0
Name of University             108
Location                       294
No of student                  132
No of student per staff        133
International Student          132
Female:Male Ratio              213
OverAll Score                  542
Teaching Score                 542
Research Score                 542
Citations Score                542
Industry Income Score          542
International Outlook Score    542
dtype: int64


Teniendo en cuenta los valores nulos para limpiar los datos, limpamos los mismos ya que, si no tenemos informacion de la universidad en cuestion no podemos clasificarlas, por ende eliminamos estas entradas, habiendo copiado previamente el dataset en una varaible "copia" para no modificar el frame original.

In [8]:
copia_df = df.copy()
copia_df.dropna(subset=['Name of University'], inplace=True)
copia_df.dropna(subset=['No of student'], inplace=True)
copia_df.dropna(subset=['Location'], inplace=True)
copia_df.dropna(subset=['No of student per staff'], inplace=True)
copia_df.dropna(subset=['Female:Male Ratio'], inplace=True)
copia_df.dropna(subset=['OverAll Score'], inplace=True)

Una vez eliminados las entradas con alguna columna nula, verificamos si todo esta correctamente.

In [ ]:
nulos_por_columna = copia_df.isnull().sum()

print(nulos_por_columna)

## Modificando nombre de columnas

Para evitar los espacios en variables y nombres muy largos modificamos el nombre de cada columna usando palabras sencillas o iniciales usando CamelCase

In [ ]:
copia_df.columns

In [10]:
copia_df.rename(columns={'University Rank':'rank', 'Name of University':'name', 'Location':'location', 'No of student':'studentCount',
       'No of student per staff':'studentStaff', 'International Student':'interStudent', 'Female:Male Ratio':'ratio',
       'OverAll Score':'overall', 'Teaching Score':'teachScore', 'Research Score':'researchScore', 'Citations Score':'citiationScore',
       'Industry Income Score':'indIncome', 'International Outlook Score':'interScore'}, inplace=True)

Tambien observamos que la la variable o la columna ratio no es atomica y esta lo que indica es que porcentaje de hombres-mujeres existen en la universidad por lo tanto procedemos a separar dichas columnas y eliminar la columna inicial con el comando drop().

In [12]:
copia_df[['maleRatio', 'femaleRatio']] = copia_df['ratio'].str.split(':', expand=True)
copia_df.drop(columns=['ratio'], inplace=True)


Verificamos que el paso anterior se ejecuto correctamente con el comando head() y posteriormente procedemos a verificar los tipos de cada variables.

In [13]:
copia_df.head()

,rank,name,location,studentCount,studentStaff,interStudent,overall,teachScore,researchScore,citiationScore,indIncome,interScore,maleRatio,femaleRatio
0,1,University of Oxford,United Kingdom,"20,965",10.6,42%,96.4,92.3,99.7,99.0,74.9,96.2,48,52
1,2,Harvard University,United States,"21,887",9.6,25%,95.2,94.8,99.0,99.3,49.5,80.5,50,50
2,3,University of Cambridge,United Kingdom,"20,185",11.3,39%,94.8,90.9,99.5,97.0,54.2,95.8,47,53
3,3,Stanford University,United States,"16,164",7.1,24%,94.8,94.2,96.7,99.8,65.0,79.8,46,54
4,5,Massachusetts Institute of Technology,United States,"11,415",8.2,33%,94.2,90.7,93.6,99.8,90.9,89.3,40,60


In [ ]:
copia_df.dtypes

Observamos que hay problemas con los datos, por ejemplo con la varible "studentCount" que muestra la cantidad de estudiantes por universidad, es de tipo object y al ser un numero su tipo de dato deberia ser entero o en su defecto flotante, se procede a corregir todos los errores de este tipo de indole.

Y procedemos a verificar que los cambios se hicieron correctamente

In [15]:
# 1. Limpiar strings
copia_df['studentCount'] = copia_df['studentCount'].str.replace(',', '').astype(float)
copia_df['interStudent'] = pd.to_numeric(copia_df['interStudent'].str.replace('%', ''), errors='coerce') / 100
copia_df['overall'] = pd.to_numeric(copia_df['overall'], errors='coerce')
copia_df['rank'] = pd.to_numeric(copia_df['rank'], errors='coerce')
# 4. Limpiar ratios (si son números limpios en formato texto)
copia_df['maleRatio'] = pd.to_numeric(copia_df['maleRatio'], errors='coerce')
copia_df['femaleRatio'] = pd.to_numeric(copia_df['femaleRatio'], errors='coerce')
print(copia_df.dtypes)

rank              float64
name               object
location           object
studentCount      float64
studentStaff      float64
interStudent      float64
overall           float64
teachScore        float64
researchScore     float64
citiationScore    float64
indIncome         float64
interScore        float64
maleRatio           int64
femaleRatio         int64
dtype: object


## Escalar valores

Como existen valores muy desproporcionados, teniendo en cuenta el numero de estudiantes y el numero de estudiantes por staff, lo mejor sera escalar o normalizar estos valores para que el modelado no arroje resultados erroneos por el hecho de que existan mas estudiantes nacionales, esto para tambien tomar en cuenta todos los valores que aporten informacion o datos al modelado.


Teniendo en cuenta que la columna "Rank" no se escala ya que es un valor asignado a esa universidad.


In [21]:
from sklearn.preprocessing import StandardScaler

escalar = [
    'studentCount', 'studentStaff', 'overall',
    'teachScore', 'researchScore', 'citiationScore',
    'indIncome', 'interScore'
]

scaler = StandardScaler()

copia_df[escalar] = scaler.fit_transform(copia_df[escalar])

print(copia_df.head())

   rank                                   name        location  studentCount  \
0   1.0                   University of Oxford  United Kingdom     -0.063906   
1   2.0                     Harvard University   United States     -0.031840   
2   3.0                University of Cambridge  United Kingdom     -0.091033   
3   3.0                    Stanford University   United States     -0.230878   
4   5.0  Massachusetts Institute of Technology   United States     -0.396041   

   studentStaff  interStudent   overall  teachScore  researchScore  \
0     -0.706077          0.42  2.481148    5.189694       4.871378   
1     -0.791837          0.25  2.378535    5.386073       4.827590   
2     -0.646045          0.39  2.344330    5.079721       4.858868   
3     -1.006236          0.24  2.344330    5.338942       4.683715   
4     -0.911900          0.33  2.293023    5.064011       4.489796   

   citiationScore  indIncome  interScore  maleRatio  femaleRatio  
0        1.858205   2.191881   

## Ultimando preparaciones
Teniendo los datos estructurados como esperamos nos damos cuenta de un detalle y es que el rank de la universidades es un dato numero PERO existen obeservacion con el valor "reporter" esto significa que las universidades mandaron su postulacion y por algun motivo no pasaron la verificacion y no le asignaron un rango, por lo tanto existe "dos tipos" de dataset, hago enfasis en dos tipos ya que esto lo podemos dividir para diferentes tipos de modelado por esto se procede a separar las universades en "df_con_nota" y "df_reporter"

In [22]:
copia_df.head()

,rank,name,location,studentCount,studentStaff,interStudent,overall,teachScore,researchScore,citiationScore,indIncome,interScore,maleRatio,femaleRatio
0,1.0,University of Oxford,United Kingdom,-0.063906,-0.706077,0.42,2.481148,5.189694,4.871378,1.858205,2.191881,2.306408,48,52
1,2.0,Harvard University,United States,-0.031840,-0.791837,0.25,2.378535,5.386073,4.827590,1.869129,0.299755,1.587085,50,50
2,3.0,University of Cambridge,United Kingdom,-0.091033,-0.646045,0.39,2.344330,5.079721,4.858868,1.785383,0.649873,2.288081,47,53
3,3.0,Stanford University,United States,-0.230878,-1.006236,0.24,2.344330,5.338942,4.683715,1.887335,1.454399,1.555013,46,54
4,5.0,Massachusetts Institute of Technology,United States,-0.396041,-0.911900,0.33,2.293023,5.064011,4.489796,1.887335,3.383771,1.990272,40,60


In [23]:
# 1. Dataset de universidades SIN calificación (las que dicen "Reporter")
df_reporter = copia_df[copia_df['rank'].isna()].copy()

# 2. Dataset de universidades CON calificación (todo lo que NO sea "Reporter")
df_con_nota = copia_df[copia_df['rank'] != 'Reporter'].copy()

#Conclusion

habiendo preprocesado los datos, ahora tenemos dos datasets separados por universidades con ranking y universidades sin rank, como se menciono anteriormente estos datasets pueden ser usados para:

df_con_nota: Este data set puede ser usados para modelados del tipo regresion

df_reporter: Este data set puede ser usados para modelados del tipo clasificacion